# Load GEMD files in database
This code is for information purposes onlny as it requires permissions on our SciServer system that users of this repo will not in general posess. The database schema is defined by files in the database_design subfolder.

#### NB parts of this code can only be run by someone with direct credentials to the database

In [ ]:
import glob
import json
import random
import pandas
import pprint
import sys
import json
import numpy as np
import pymssql

In [ ]:
import sqlalchemy as sqla
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import select

In [ ]:
from gemd_database import MSSQLDatabase

In [ ]:
with open('/home/idies/workspace/persistent/.private/gemd.json','r') as f:
    AUTH = json.load(f)
GEMDDB=MSSQLDatabase(AUTH,'GEMD')

# Load in GEMD database
Prepared with 
```sql
create table GEMDModel (
    id bigint identity(1,1) not null
  , name varchar(1024) not null
  , constraint pk_gemdmodel primary key(id)
)

create table GEMDContext (
  uid varchar(64) not null
  , model_id bigint not null
  , gemd_type varchar(32) not null
  , context varchar(max) not null
  , constraint pk_GEMDContext primary key(uid)
  , constraint fk_gemdcontext_model foreign key (model_id) REFERENCES gemdmodel(id)
  )
```

In [ ]:
ddl = """
create table GEMDModel (
    id bigint identity(1,1) not null
  , name varchar(1024) not null
  , constraint pk_gemdmodel primary key(id)
)

create table GEMDContext (
  uid varchar(64) not null
  , model_id bigint not null
  , gemd_type varchar(32) not null
  , context varchar(max) not null
  , constraint pk_GEMDContext primary key(uid)
  , constraint fk_gemdcontext_model foreign key (model_id) REFERENCES gemdmodel(id)
  )
  
 CREATE TABLE GEMDEdge(
    id bigint IDENTITY(1,1)  not null
  , from_uid varchar(64) not null
  , to_uid varchar(64) not null
  , gemd_ref varchar(64) not null
)
"""
# GEMDDB.execute_update(ddl)

In [ ]:
GEMDDB.execute_update("drop table gemdobject")

In [ ]:
sql = "select * from information_schema.tables "
GEMDDB.execute_query(sql)

In [ ]:
Base = automap_base()
# Base.prepare(GEMDDB.ENGINE, reflect=True)
Base.prepare(GEMDDB.ENGINE)#, reflect=True)
for c in Base.classes:
    print(c)
GEMDContext = Base.classes.GEMDContext
GEMDModel  = Base.classes.GEMDModel

## create object files from the JSON GEMD 
complete_data_model.json is located in an aptly named folder and contains all JSON content to be loaded

In [ ]:
folders = glob.glob('/home/idies/workspace/Temporary/gerard/GEMD-Maggie/GEMD_models/*')

In [ ]:
%%time
models = []
for folder in folders:
    model = GEMDModel(name=folder.split("/")[-1])
    models.append(model)
    files = glob.glob(f"{folder}/*")
    for file in files:
        with open(f'{file}','r') as f:
            try:
                d=json.load(f)
                GEMDContext(gemd_type=d['type'],uid=d['uids']['auto'],context=json.dumps(d),gemdmodel=model)
            except Exception as e:
                print("ERROR:",file)
                print(e)

In [ ]:
%%time
session = Session(GEMDDB.ENGINE)
for model in models:
    session.add(model)
session.commit()

Here starts some code that is only runnable by someone who has direct credentials to the database

# add more explicit tables and views
For each of the main object types create table/view and extract somewhat more explicit metadata into columns. In particular a name and all the references/foreign keys to other tables n this set. Note that the tables are defined implicitly using the <tt>SELECT ... INTO...</tt> supported by MS SQL. For this reason one needs a database user with appopriate privileges.

<img src='https://citrineinformatics.github.io/gemd-docs/img/GEMD-Object-Graph.png'></img>


Use class in utility script. Again must be laoded with appropriate permissions not avilable to random users.

## material

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      context
  into material_template
  from gemdcontext
 where gemd_type='material_template'
"""
r=GEMDDB.execute_update(sql)    

In [ ]:
sql="""
--drop table material_spec
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      json_value(context,'$.process.id') as process_spec_uid
,      context
  into material_spec
  from gemdcontext
 where gemd_type='material_spec'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.spec.id') as spec_uid
,      json_value(context,'$.process.id') as process_run_uid
,      context
  into material_run
  from gemdcontext
 where gemd_type='material_run'
"""
GEMDDB.execute_update(sql)

## measurement

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      context
  into measurement_template
  from gemdcontext
 where gemd_type='measurement_template'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      context
  into measurement_spec
  from gemdcontext
 where gemd_type='measurement_spec'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.source.performed_by') as performed_by
,      json_value(context,'$.source.performed_date') as performed_date
,      json_value(context,'$.spec.id') as spec_uid
,      json_value(context,'$.material.id') as material_uid
,      context
  into measurement_run
  from gemdcontext
 where gemd_type='measurement_run'
"""
GEMDDB.execute_update(sql)

## process

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      context
  into process_template
  from gemdcontext
 where gemd_type='process_template'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.source.performed_by') as p_performed_by
,      json_value(context,'$.source.performed_date') as p_performed_date
,      json_value(context,'$.spec.id') as spec_uid
,      context
  into process_run
  from gemdcontext
 where gemd_type='process_run'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      model_id
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      context
  into process_spec
  from gemdcontext
 where gemd_type='process_spec'
"""
GEMDDB.execute_update(sql)

## ingredient

In [ ]:
# into ingredient_spec
#   
sql="""
select distinct s.gemd_type as spec
,      s.model_id
, m.gemd_type as material, p.gemd_type as process
  from gemdcontext i 
  join gemdcontext s on s.uid=json_value(i.context,'$.spec.id')
  join gemdcontext m on m.uid=json_value(i.context,'$.material.id')
  join gemdcontext p on p.uid=json_value(i.context,'$.process.id')
 where i.gemd_type='ingredient_run'
"""
GEMDDB.execute_query(sql)
# GEMDDB.execute_update(sql)

In [ ]:
# 
#   
sql="""
select i.uid
,      model_id
,      json_value(i.context,'$.name') as name
,      json_value(i.context,'$.material.id') as material_spec_uid
,      json_value(i.context,'$.process.id') as process_spec_uid
,      i.context
  into ingredient_spec
  from gemdcontext i 
where i.gemd_type='ingredient_spec'
"""
# GEMDDB.execute_query(sql)
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select i.uid
,      model_id
,      json_value(i.context,'$.name') as name
,      json_value(i.context,'$.spec.id') as spec_uid
,      json_value(i.context,'$.material.id') as material_run_uid
,      json_value(i.context,'$.process.id') as process_run_uid
,      i.context
  into ingredient_run
  from gemdcontext i 
where i.gemd_type='ingredient_run'
"""
# GEMDDB.execute_query(sql)
GEMDDB.execute_update(sql)

## attribute_template
generic for ['condition_template','property_template','parameter_template']

in following SQL, iso JSON_QUERY could use JSON_OBJECT in SQL Server 2022 and later

In [ ]:
sql = """
select gc.uid, gc.model_id, gc.gemd_type
,      b.name, b.description
,      b.bounds_type, b.units
,      case when bounds_type = 'real_bounds' or bounds_type = 'integer_bounds' then
              JSON_QUERY((
              select lower_bound = JSON_VALUE(b.bounds,'$.lower_bound')
              ,      upper_bound = JSON_VALUE(b.bounds,'$.upper_bound')
                for json path
              ))
            when bounds_type = 'categorical_bounds' then
              JSON_QUERY(b.bounds,'$.categories')
            when bounds_type = 'composition_bounds' then
              JSON_QUERY(b.bounds,'$.components')
            else null end as bounds
,      b.tags
,      gc.context
into attribute_template
  from GEMDContext gc
  cross apply OPENJSON(gc.context,'$') with (
    name nvarchar(128) '$.name'
  , description nvarchar(max) '$.description'
  , bounds_type nvarchar(32) 'lax$.bounds.type'
  , units nvarchar(32) 'lax$.bounds.default_units'
  , tags nvarchar(max) 'lax$.bounds.default_units' as json
  , bounds nvarchar(max) 'lax$.bounds' as json) b
 where gc.gemd_type in ('condition_template','property_template','parameter_template')
"""
GEMDDB.execute_update(sql)

# attributes: parameter, condition, property

In [ ]:
sql="""
with a as (
select s.uid, s.gemd_type
,      sp.*
  from GEMDContext s
  cross apply OPENJSON(s.context,'$.parameters') with (
     template_id nvarchar(128) '$.template.id',
     name nvarchar(128) '$.name',
     value_type nvarchar(max) '$.value.type' ,
     value_units nvarchar(max) '$.value.units' ,
     value nvarchar(max) '$.value' as json,
     attribute_type nvarchar(128) '$.type',
     parameter nvarchar(max) '$' as json) sp
)
select a.uid as gemdcontext_uid, a.template_id, a.name
,      case when a.value_type = 'nominal_categorical' then json_value(a.value,'$.category')
            when a.value_type = 'nominal_real' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_integer' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_composition' then json_value(a.value,'$')
       else NULL end as value
       , a.value_type
       , a.value_units
,      a.attribute_type
,      a.parameter
into parameter
  from a 
"""
GEMDDB.execute_update(sql)

In [ ]:
sql = """
with c as (
select s.uid, s.gemd_type, sp.property_type
,      case when sp.property_type = 'property_and_conditions' then sp.p_property
            else sp.property end property
,      case when sp.property_type = 'property_and_conditions' then sp.p_conditions
            else NULL end conditions
  from GEMDContext s
  cross apply OPENJSON(s.context,'$.properties') with (
     property_type nvarchar(128) '$.type',
     p_property nvarchar(max) '$.property' as json,
     p_conditions nvarchar(max) '$.conditions' as json,
     property nvarchar(max) '$' as json) sp
), a as (
select c.uid,c.gemd_type, c.property_type, c.property
,      case when c.conditions = '[]' then NULL else c.conditions end as conditions
,      sp.*
  from c
  cross apply OPENJSON(c.property,'$') with (
     template_id nvarchar(128) '$.template.id',
     name nvarchar(128) '$.name',
     value_type nvarchar(max) '$.value.type' ,
     value_units nvarchar(max) '$.value.units' ,
     value nvarchar(max) '$.value' as json) sp
)
select a.uid as gemdcontext_uid, a.template_id, a.name
,      case when a.value_type = 'nominal_categorical' then json_value(a.value,'$.category')
            when a.value_type = 'nominal_real' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_integer' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_composition' then json_value(a.value,'$')
       else NULL end as value
,      a.value_type
,      a.value_units
,      a.property_type as attribute_type
,      a.property
,      a.conditions
  into property
  from a 
"""
GEMDDB.execute_update(sql)

In [ ]:
sql = """
with a as (
select s.uid, s.gemd_type
,      sp.*
  from GEMDContext s
  cross apply OPENJSON(s.context,'$.conditions') with (
     template_id nvarchar(128) '$.template.id',
     name nvarchar(128) '$.name',
     value_type nvarchar(max) '$.value.type' ,
     value_units nvarchar(max) '$.value.units' ,
     value nvarchar(max) '$.value' as json,
     attribute_type nvarchar(128) '$.type',
     condition nvarchar(max) '$' as json) sp
)
select a.uid as gemdcontext_uid, a.template_id, a.name
,      case when a.value_type = 'nominal_categorical' then json_value(a.value,'$.category')
            when a.value_type = 'nominal_real' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_integer' then json_value(a.value,'$.nominal')
            when a.value_type = 'nominal_composition' then json_value(a.value,'$')
       else NULL end as value
       , a.value_type
       , a.value_units
,      a.attribute_type
,      a.condition
into condition
  from a 
"""
GEMDDB.execute_update(sql)

In [ ]:
sql = """
create or alter view attribute as
select * from property
union
select *, null from condition
union
select *, null from parameter
"""
GEMDDB.execute_update(sql)

## graph table
adding edges to contexts

In [ ]:
sql = """
-- delete from GEMDEdge
insert into GEMDEdge 
select uid,material_run_uid, 'ingredient_run:material_run'
from ingredient_run
where material_run_uid is not null
union
select uid,process_run_uid, 'ingredient_run:process_run'
from ingredient_run
where process_run_uid is not null
union
select uid,spec_uid, 'ingredient_run:spec'
from ingredient_run
where process_run_uid is not null
union
select uid,process_spec_uid, 'ingredient_spec:process_spec'
from ingredient_spec
where process_spec_uid is not null
union
select uid,material_spec_uid, 'ingredient_spec:material_spec'
from ingredient_spec
where material_spec_uid is not null
union
select uid,spec_uid, 'material_run:spec'
from material_run 
where spec_uid is not null
union
select uid, process_run_uid, 'material_run:process_run'
from material_run 
where process_run_uid is not null
union
select uid,process_spec_uid, 'material_spec:process_spec'
from material_spec 
where process_spec_uid is not null
union
select uid,template_uid, 'material_spec:template'
from material_spec
where template_uid is not null
union
select uid,spec_uid, 'measurement_run:spec'
from measurement_run
where spec_uid is not null
union
select uid,material_uid, 'measurement_run:material'
from measurement_run
where material_uid is not null
union
select uid,template_uid, 'measurement_spec:template'
from measurement_spec
where template_uid is not null
union
select uid,spec_uid, 'process_run:spec'
from process_run
where spec_uid is not null
union
select uid,template_uid, 'process_spec:template'
from process_spec
where template_uid is not null
"""
GEMDDB.execute_update(sql)

# some more
Find gemd_types not yet representes

In [ ]:
sql="""
select top 1000 s.uid, s.gemd_type
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(sp.property,'$.name') as p
,      sp.property
,      JSON_VALUE(sp.property,'$.template.id') as template_uid
,      pt.*
  from GEMDContext s
  cross apply OPENJSON(s.context,'$.properties') with (property nvarchar(max) 'lax$.property' as json) sp
  join attribute_template pt on pt.uid = JSON_VALUE(sp.property,'$.template.id')
 order by 2,1
  """
df=GEMDDB.execute_query(sql).convert_dtypes()
df

In [ ]:
sql="""
select top 1000 s.uid, s.gemd_type
,      JSON_VALUE(s.context,'$.name') as spec
,      JSON_VALUE(sp.property,'$.name') as p
,      sp.property
,      JSON_VALUE(sp.property,'$.template.id') as template_uid
,      pt.*
  from GEMDContext s
  cross apply OPENJSON(s.context,'$.conditions') with (condition nvarchar(max) 'lax$' as json) sp
  join attribute_template pt on pt.uid = JSON_VALUE(sp.property,'$.template.id')
 order by 2,1
  """
df=GEMDDB.execute_query(sql).convert_dtypes()
df

In [ ]:
props

In [ ]:
sql="""
with gt as (
select distinct gemd_type from GEMDContext
)
select gt.gemd_type
  from  gt
  where not exists (Select t.table_name from information_Schema.tables t where t.table_name=gt.gemd_type)
order by 1
"""
GEMDDB.execute_query(sql)

In [ ]:
sql="""
with some_specs as (  
select top 10 *              -- using SELECT TOP N ... ORDER BY newid() is standard trick for getting a random sample of N entries
  from gemdcontext
 where gemd_type='process_spec' 
order by newid()                       
)
select s.uid as spec_id
,      json_value(s.context,'$.name') as spec
,      r.uid as run_id
,      r.context as run
  from some_specs s
  join gemdcontext r
 on r.gemd_type='process_run' 
   and JSON_VALUE(r.context,'$.spec.id')=s.uid
order by spec
"""
cj.executeQuery(sql,DATABASE)

In [ ]:
sql = """ 
select top 10 *
  from gemdcontext 
  where context like '%00f132e7-60d8-472c-80cc-0aec952f214e%'
"""
df = GEMDDB.execute_query(sql)
# [json.loads(c) for c in df['context']]
df

In [ ]:
[{'bounds': {'default_units': 'meter / second',
   'lower_bound': 0.0,
   'type': 'real_bounds',
   'upper_bound': 3000.0},
  'description': 'Maximum velocity measured using PDV',
  'name': 'Max Velocity',
  'tags': [],
  'type': 'property_template',
  'uids': {'auto': '00f132e7-60d8-472c-80cc-0aec952f214e'}}]

In [ ]:
contexts